In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('../input/real-estate-dataset/data.csv')

In [ ]:
df.head()

**Preprocess**

In [ ]:
df.isna().sum()

In [ ]:
df['RM']=df['RM'].fillna(df['RM'].mean())

In [ ]:
df.shape

In [ ]:
{columns : len(df[columns].unique()) for columns in df.select_dtypes('int')}

In [ ]:
df['CHAS'].value_counts()

In [ ]:
sns.pairplot(df)

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns

plt.figure(figsize=(16,10))
for i in range (len(df.columns)):
    plt.subplot(3,5,i+1)
    sns.boxplot(df[df.columns[i]])

In [ ]:
from scipy import stats

def find_outliers(df, threshold):
    
    threshold_z_score = stats.norm.ppf(threshold)
    
    z_score = pd.DataFrame(np.abs(stats.zscore(df)), columns = df.columns)
    
    return (z_score > threshold_z_score). sum(axis=0)
    

In [ ]:
stats.norm.ppf(0.5)

In [ ]:
find_outliers(df, 0.9999)

In [ ]:
def remove_outliers(df, threshold):
    
    threshold_z_score = stats.norm.ppf(threshold)
    
    z_score = pd.DataFrame(np.abs(stats.zscore(df)), columns = df.columns)
    
    z_score = (z_score > threshold_z_score)
    
    # get the indecies
    
    outliers =  z_score.sum(axis=1)
    outliers = outliers > 0
    outliers_index = df.index[outliers]
    
    df =df.drop(outliers_index, axis =0).reset_index(drop=True)
    
    return df
    

In [ ]:
df=remove_outliers(df,0.9999)

In [ ]:
df.shape

In [ ]:
x = df.drop('CHAS', axis =1)

In [ ]:
y = df['CHAS']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y, train_size = 0.7, random_state = 1)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(x_train)

x_train = pd.DataFrame(sc.transform(x_train),columns = x.columns)
x_test = pd.DataFrame(sc.transform(x_test),columns = x.columns) 

In [ ]:
# treating imbalanced dataset 

from imblearn import over_sampling, under_sampling
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state = 33)

In [ ]:
x_train_smote, y_train_smote = smote.fit_sample(x_train,y_train)

In [ ]:
import seaborn as sns
sns.countplot(y_train_smote)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
Knn = KNeighborsClassifier()
Knn.fit(x_train_smote, y_train_smote)

In [ ]:
print(Knn.score(x_train_smote, y_train_smote))
print(Knn.score(x_test,y_test))

In [ ]:
predict = Knn.predict(x_test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [ ]:
print( confusion_matrix(y_test, predict))

In [ ]:
print(classification_report(y_test, predict))

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.svm import SVC

In [ ]:
parameter = [
  {'C': [0.01, 0.05, 0.5, 1], 'kernel': ['linear']},
  {'C': [0.01, 0.05, 0.5, 1], 'kernel': ['rbf']},
]

# Create a classifier object with the classifier and parameter candidates
GSCV = GridSearchCV(estimator=svm.SVC(), param_grid=parameter, n_jobs=-1)

# Train the classifier on data1's feature and target data
GSCV.fit(x_train_smote, y_train_smote)

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

kfold = KFold(n_splits=10, random_state=10)
score = cross_val_score(GSCV,x_train_smote, y_train_smote, cv=kfold, scoring='accuracy')
print(score)
print(score.mean())

In [ ]:
print('Best score :', GSCV.best_score_)

In [ ]:
print('Best C:',GSCV.best_estimator_.C) 
print('Best Kernel:',GSCV.best_estimator_.kernel)